In [1]:
%matplotlib inline
import qiime2 as q2

from biom import Table

import pandas as pd, numpy as np, seaborn as sns
from skbio.stats.distance import mantel, DistanceMatrix
from qiime2.plugins import diversity

In [2]:
cd trimmed-150nts/

/Users/yoshikivazquezbaeza/Documents/PDF/KnightLaboratory/HastyWater/trimmed-150nts


Do the communities remain the same after the normalization takes place? To check this we can compare a number of distance metrics using Mantel's test.

In [22]:
results = []
metrics = ['jaccard', 'braycurtis', 'canberra', 'unweighted_unifrac', 'weighted_unifrac']

table_normalized = q2.Artifact.load('feature-table.even.33404.qza')
table_unnormalized = q2.Artifact.load('feature-table.even.18813.normalized.qza')

for metric in metrics:
    
    if 'unifrac' in metric:
        dm_norm, = diversity.methods.beta_phylogenetic_alt(table_normalized, tree, metric)
        dm_unorm, = diversity.methods.beta_phylogenetic_alt(table_unnormalized, tree, metric)
    else:
        dm_norm, = diversity.methods.beta(table_normalized, metric)
        dm_unorm, = diversity.methods.beta(table_unnormalized, metric)

    results.append(mantel(dm_norm.view(DistanceMatrix), dm_unorm.view(DistanceMatrix)))
    
summary = pd.DataFrame(columns=['r**2', 'p-value', 'Sample Size per Matrix'], data=results, index=metrics)

/Users/yoshikivazquezbaeza/miniconda/envs/qiime2-2018.4/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [24]:
summary.sort_values(by='r**2')

,r**2,p-value,Sample Size per Matrix
canberra,0.857417,0.001,118
jaccard,0.970628,0.001,118
unweighted_unifrac,0.977602,0.001,118
braycurtis,0.985944,0.001,118
weighted_unifrac,0.992905,0.001,118


For the most part the distances are very correlated and the tables don't differ much between them, since most metrics treat the data as proportions or on a qualitative fashion. Of importance, the Canberra distance differ's the most between tables.